In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import sys,os,re,csv,glob,collections,itertools,pickle,datetime
import dateutil
import igraph
import shapefile
import numpy.ma as ma
import matplotlib
import scipy
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as mcolors
import gdal
import scipy.io
import matplotlib.image as mpimg

from scipy import ndimage
from scipy.spatial import ConvexHull
from sklearn.decomposition import FastICA as ICA
from sklearn.decomposition import PCA
from scipy.ndimage.filters import gaussian_filter as gaussfilter
from sklearn.neighbors import KDTree
from mapbox import Static
from tqdm import tqdm
from gdalconst import *
from scipy.misc import imresize
from scipy.misc import imsave
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, zoomed_inset_axes
from shapely.geometry import Point as SPoint
from shapely.geometry.polygon import Polygon as SPolygon
from statsmodels.tsa.stattools import grangercausalitytests as granger

### Build Distance Grid

In [2]:
[X,Y] = np.meshgrid(range(512),range(512))
X = X.reshape((512*512))
Y = Y.reshape((512*512))
C = np.zeros((X.shape[0],2))
C[:,0] = X
C[:,1] = Y

### Loop

In [4]:
for imCount in tqdm(range(8000)):

    #Load Image
    filename = '../../data/OSMNet/raw/label/' + '/' + str(int(imCount)) + '.png'
    img = mpimg.imread(filename)
    
    #Find Boundaries
    D = ndimage.grey_dilation(img, size=(3, 3), structure=np.ones((3, 3)))
    boundaries = (D-img) - 1

    #Remove Stray Points 1
    sat = np.pad(boundaries, pad_width=1, mode='constant', constant_values=0)
    sat = np.cumsum(np.cumsum(sat, axis=0), axis=1)
    sat = np.pad(sat, ((1, 0), (1, 0)), mode='constant', constant_values=0)
    sum3x3 = sat[3:, 3:] + sat[:-3, :-3] - sat[3:, :-3] - sat[:-3, 3:]
    sum3x3 -= boundaries
    boundaries[sum3x3 == 0] = 0

    #Remove Stray Points 2
    sat = np.pad(boundaries, pad_width=1, mode='constant', constant_values=0)
    sat = np.cumsum(np.cumsum(sat, axis=0), axis=1)
    sat = np.pad(sat, ((1, 0), (1, 0)), mode='constant', constant_values=0)
    sum3x3 = sat[3:, 3:] + sat[:-3, :-3] - sat[3:, :-3] - sat[:-3, 3:]
    sum3x3 -= boundaries
    boundaries[sum3x3 == 0] = 0  
    
    #Get Boundary Coordinates
    x,y = np.where(boundaries == 1)
    B = np.zeros((len(x),2))
    B[:,0] = x
    B[:,1] = y
    
    #Build KD Tree
    tree = KDTree(B, leaf_size=2)
    
    #Query
    dist,ind = tree.query(C,1)
    dist = np.array(dist)
    dist = dist.reshape((512,512))
    dist = dist.T    

    #Create Signed Image
    newimg = (img * 2) - 1
    sigdis = dist*newimg
    
    #Bin Values
    val = np.max(sigdis)
    sigdis[sigdis < -val] = -val
    sigdis = sigdis + val
    sigdis = sigdis / (val*2)
    sigdis = 64 + sigdis * 128
    sigdis = np.round(sigdis)
    sigdis = sigdis.astype(int)
    
    #Save
    filename = '../../data/OSMNet/raw/signed' + '/' + str(int(imCount)) + '.png'
    imsave(filename, sigdis)